In [231]:
import requests # type: ignore
import os
from bs4 import BeautifulSoup # type: ignore
from dotenv import load_dotenv # type: ignore
from requests.exceptions import RequestException # type: ignore
from fake_useragent import UserAgent # type: ignore
from tenacity import (retry,retry_if_exception_type,stop_after_attempt , wait_exponential)
from datetime import date ,time

In [232]:
load_dotenv()

True

In [233]:
base_url =  os.getenv("BASE_URL")
matches_url = os.getenv("MATCHES_URL")

In [234]:
def genrate_random_useragent():
    # Create a UserAgent object
    ua = UserAgent()
    # Get a random user agent string
    random_user_agent = ua.random
    return random_user_agent



In [235]:
@retry(
    reraise = True,
    stop = stop_after_attempt(5),
    wait = wait_exponential(multiplier = 1 , min = 1 , max = 10),
    retry = (retry_if_exception_type(RequestException))
)
def requests_with_retry(url):
    
    user_agent = genrate_random_useragent()
    headers = {'User-Agent': user_agent}
    
    try:
        res = requests.get(url=url,headers=headers)
        res.raise_for_status()
        print(res.request.headers)
        return res
    except RequestException as e:
        print(f"Exception occured : {e}")
        raise RequestException
           
    

In [236]:
response = requests_with_retry(url=matches_url)
soup = BeautifulSoup(response.content)


Exception occured : 403 Client Error: Forbidden for url: https://www.hltv.org/matches
Exception occured : 403 Client Error: Forbidden for url: https://www.hltv.org/matches
Exception occured : 403 Client Error: Forbidden for url: https://www.hltv.org/matches
{'User-Agent': 'Mozilla/5.0 (Linux; Android 10; K) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Mobile Safari/537.36', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}


In [237]:
upcomming_matches_sections = soup.find_all('div',class_='upcomingMatchesSection')

In [220]:
section_content = upcomming_matches_sections[0].contents

for child in upcomming_matches_sections[0].descendants:
    if not isinstance(child, str):
        print(child)
        print(child['class'])
        print((child.text))
        break

<span class="matchDayHeadline">Wednesday - 2024-05-29</span>
['matchDayHeadline']
Wednesday - 2024-05-29


In [221]:
section = upcomming_matches_sections[0]


In [ ]:
match = {
    'matchDate' : date,
    'matchTime' : time,
    'matchTeam1': str,
    'matchTeam2': str,
    'matchEventName': str,
    'matchMeta':str,
    'matchRating':int,
}

In [226]:
for child in  section.children:
    print(child)

<span class="matchDayHeadline">Wednesday - 2024-05-29</span>


<div class="upcomingMatch" data-zonedgrouping-entry-unix="1716993000000" filteraslive="false" lan="false" stars="0" team1="11814" team2="11006"><a class="match a-reset" href="/matches/2372653/corinthians-vs-9z-academy-cct-season-2-south-america-series-1">
<div class="matchInfo">
<div class="matchTime" data-time-format="HH:mm" data-unix="1716993000000">16:30</div>
<div class="matchRating"><i class="fa fa-star faded"></i><i class="fa fa-star faded"></i><i class="fa fa-star faded"></i><i class="fa fa-star faded"></i><i class="fa fa-star faded"></i></div>
<div class="matchMeta">bo3</div>
</div>
<div class="matchTeams text-ellipsis">
<div class="matchTeam team1">
<div class="matchTeamLogoContainer"><img alt="Corinthians" class="matchTeamLogo" src="https://img-cdn.hltv.org/teamlogo/Q9vtsznWEOK6yiakCWx0n2.png?ixlib=java-2.1.0&amp;w=50&amp;s=f7d33a9ee197c1753d6e6fcb0bcf55bb" title="Corinthians"/></div>
<div class="matchTeamName tex

TypeError: 'list_iterator' object is not subscriptable